In [20]:
! pip install -r requirements.txt

In [21]:
pip install requests



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install beautifulsoup4


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [23]:
import requests
from bs4 import BeautifulSoup
import csv

In [28]:
eventlist = []

for i in range(1, 41):
    url = f"https://visitseattle.org/events/page/{i}"
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        selector = "div.search-result-preview > div > h3 > a"
        a_eles = soup.select (selector)
        eventlist += [x['href']for x in a_eles]

print('Event List:', eventlist)

Event List: ['https://visitseattle.org/events/steinunn-porarinsdottir-wayfinders/', 'https://visitseattle.org/events/hanako-oleary-izanami/', 'https://visitseattle.org/events/gage-alumni-show/', 'https://visitseattle.org/events/genre-box/', 'https://visitseattle.org/events/bohemia/', 'https://visitseattle.org/events/alice-in-shadowland/', 'https://visitseattle.org/events/lee-ritenour-and-dave-grusin/', 'https://visitseattle.org/events/tacoma-home-garden-show/', 'https://visitseattle.org/events/annie-eastwood-with-kimball-and-the-fugitives/', 'https://visitseattle.org/events/black-and-boujee-panel-discussion/', 'https://visitseattle.org/events/celebrate-asia/', 'https://visitseattle.org/events/djangos-birthday-concert-with-ranger-and-the-re-arrangers/', 'https://visitseattle.org/events/family-day-with-birds-connect-seattle/', 'https://visitseattle.org/events/fossil-finders-ice-ages/', 'https://visitseattle.org/events/seattle-kraken-vs-columbus-blue-jackets/', 'https://visitseattle.org/e

In [29]:
# open a csv file to write
with open('eventlist.csv', 'w', newline = '') as f:
    #create a csv writer object
    writer = csv.writer(f)

    # write the header
    writer.writerow(["Name", "Date & Time", "Location", "Type", "Region"])

    # loop through each event and write data to csv file
    for event_url in event_list:
        res = requests.get(event_url)

        # check if the server response is ok
        if res.status_code == 200:
            event_info = []
            # parse the html text
            soup = BeautifulSoup(res.text, "html.parser")

            # get the name of the event
            name_selector = "div.medium-6.columns.event-top > h1"
            name_eles = soup.select(name_selector)
            event_info.append(name_eles[0].text)
            
            # get the date and time of the event
            date_and_time_selector = "div.medium-6.columns.event-top > h4 > span:nth-child(1)"
            date_and_time_eles = soup.select(date_and_time_selector)
            event_info.append(date_and_time_eles[0].text)

            # get the location of the event
            location_selector = "div.medium-6.columns.event-top > h4 > span:nth-child(2)"
            location_eles = soup.select(location_selector)
            event_info.append(location_eles[0].text)

            # get the type of the event
            type_selector = "div.medium-6.columns.event-top > a:nth-child(3)"
            type_eles = soup.select(type_selector)
            event_info.append(type_eles[0].text)

            # get the region of the event
            region_selector = "div.medium-6.columns.event-top > a:nth-child(4)"
            region_eles = soup.select(region_selector)
            event_info.append(region_eles[0].text)

            writer.writerow(event_info)


In [30]:
import requests
import datetime
import re

In [34]:
# read the event file into a list of event dictionaries, using the header row to determine column names.
events_file = open('event_list.csv', 'r')
events = list( csv.DictReader(events_file, delimiter=','))
events_file.close()

In [36]:
with open('event_weather_info.csv', 'w', newline = '') as f:
    # create a csv writer object
    writer = csv.writer(f)

    # write the header
    writer.writerow(["Name", "Region", "Weathers"])
    

In [37]:
import csv
import re
import requests
import datetime

def get_weather_for_region(region, event_date):
    # Look up the location coordinates
    location_response = requests.get(f"https://nominatim.openstreetmap.org/search.php?q={region}&format=jsonv2")
    if location_response.status_code == 200 and location_response.json():
        location_data = location_response.json()[0]
        lat, lon = location_data['lat'], location_data['lon']

        # Look up the weather for the location
        weather_response = requests.get(f"https://api.weather.gov/points/{lat},{lon}")
        if weather_response.status_code == 200:
            weather_data = weather_response.json()
            forecast_url = weather_data['properties']['forecast']
            forecast_response = requests.get(forecast_url)
            if forecast_response.status_code == 200:
                forecast_data = forecast_response.json()

                # Find the daytime weather for the event date
                for period in forecast_data['properties']['periods']:
                    if period['isDaytime']:
                        start_time = datetime.datetime.fromisoformat(period['startTime']).date()
                        event_date_obj = datetime.datetime.strptime(event_date, "%m/%d/%Y").date()
                        if start_time == event_date_obj:
                            return f"{period['temperature']}°F, {period['shortForecast']}"

# Read the event file
with open('event_list.csv', 'r') as events_file:
    events = list(csv.DictReader(events_file))

# Write to the new event weather info file
with open('event_weather_info.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=["Name", "Region", "Weathers"])
    writer.writeheader()
    for event in events:
        date = event['Date & Time']
        if re.match(r'^\d{1,2}/\d{1,2}/\d{4}$', date):
            weather_info = get_weather_for_region(event['Region'], date)
            if weather_info:
                writer.writerow({
                    "Name": event['Name'],
                    "Region": event['Region'],
                    "Weathers": weather_info
                })
